# RNN text sentiment analyzer

In [7]:
from data_processing.text_datasets import get_poem_sentiment_dataset
import torch
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

In [8]:
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)
tokenizer = gensim_model.wv.key_to_index

def tokenize_function(examples):
    review_tokenized = []
    all_parsed = 0
    unknows = 0
    for word in examples.split():
        all_parsed+=1
        try:
            review_tokenized.append(tokenizer[word.lower()])
        except:
            unknows +=1
    return review_tokenized

[==================================================] 100.0% 31.6/31.6MB downloaded


In [9]:
train_dataset, validation_dataset, test_dataset = get_poem_sentiment_dataset(tokenize_function)

Train dataset size: 892 (tensor([  23, 8638,  758,    4,   65, 5629]), tensor(1))
